In [1]:
from pathlib import Path
import numpy as np
import torch
from typing import List
from torch.nn.utils.rnn import pad_sequence
from mltrainer import rnn_models, Trainer
from torch import optim

from mads_datasets import datatools
import mltrainer
mltrainer.__version__

'0.2.5'

In [2]:
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import PaddedPreprocessor
preprocessor = PaddedPreprocessor()

gesturesdatasetfactory = DatasetFactoryProvider.create_factory(DatasetType.GESTURES)
streamers = gesturesdatasetfactory.create_datastreamer(batchsize=32, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]

2025-10-04 21:48:15.690 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at C:\Users\mwien\.cache\mads_datasets\gestures
100%|██████████| 651/651 [00:00<00:00, 3165.53it/s]


In [3]:
trainstreamer = train.stream()
validstreamer = valid.stream()
x, y = next(iter(trainstreamer))

In [4]:
from mltrainer import TrainerSettings, ReportTypes
from mltrainer.metrics import Accuracy

accuracy = Accuracy()


In [5]:
model = rnn_models.BaseRNN(
    input_size=3,
    hidden_size=64,
    num_layers=1,
    horizon=20,
)

In [6]:
yhat = model(x)

In [7]:
loss_fn = torch.nn.CrossEntropyLoss()

In [8]:
import torch
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
    print("Using MPS")
elif torch.cuda.is_available():
    device = "cuda:0"
    print("using cuda")
else:
    device = "cpu"
    print("using cpu")

# on my mac, at least for the BaseRNN model, mps does not speed up training
# probably because the overhead of copying the data to the GPU is too high
# so i override the device to cpu
device = "cpu"
# however, it might speed up training for larger models, with more parameters

using cpu


Set up the settings for the trainer and the different types of logging you want

In [9]:
settings = TrainerSettings(
    epochs=16, # increase this to about 100 for training
    metrics=[accuracy],
    logdir=Path("gestures"),
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TOML, ReportTypes.TENSORBOARD, ReportTypes.MLFLOW],
    scheduler_kwargs={"factor": 0.5, "patience": 5},
    earlystop_kwargs = {
        "save": False, # save every best model, and restore the best one
        "verbose": True,
        "patience": 5, # number of epochs with no improvement after which training will be stopped
        "delta": 0.0, # minimum change to be considered an improvement
    }
)
settings

epochs: 16
metrics: [Accuracy]
logdir: gestures
train_steps: 81
valid_steps: 20
reporttypes: [<ReportTypes.TOML: 'TOML'>, <ReportTypes.TENSORBOARD: 'TENSORBOARD'>, <ReportTypes.MLFLOW: 'MLFLOW'>]
optimizer_kwargs: {'lr': 0.001, 'weight_decay': 1e-05}
scheduler_kwargs: {'factor': 0.5, 'patience': 5}
earlystop_kwargs: {'save': False, 'verbose': True, 'patience': 5, 'delta': 0.0}

In [10]:
import torch.nn as nn
import torch
from torch import Tensor
from dataclasses import dataclass

@dataclass
class ModelConfig:
    input_size: int
    hidden_size: int
    num_layers: int
    output_size: int
    dropout: float = 0.0

class GRUmodel(nn.Module):
    def __init__(
        self,
        config,
    ) -> None:
        super().__init__()
        self.config = config
        self.rnn = nn.GRU(
            input_size=config.input_size,
            hidden_size=config.hidden_size,
            dropout=config.dropout,
            batch_first=True,
            num_layers=config.num_layers,
        )
        self.linear = nn.Linear(config.hidden_size, config.output_size)

    def forward(self, x: Tensor) -> Tensor:
        x, _ = self.rnn(x)
        last_step = x[:, -1, :]
        yhat = self.linear(last_step)
        return yhat

In [ ]:
import mlflow
from datetime import datetime

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("gestures")
modeldir = Path("gestures").resolve()
if not modeldir.exists():
    modeldir.mkdir(parents=True)
    
n_repeats = 3

for repeat in range(n_repeats):
    print(f"Repeat {repeat+1}/{n_repeats}")
    
    with mlflow.start_run():
        # Set MLflow tags to record metadata about the model and developer
        mlflow.set_tag("model", f"{repeat}_GRU_16epochs")
        mlflow.set_tag("dev", "Marcello")
        # Log hyperparameters to MLflow

        mlflow.log_param("epochs", settings.epochs)

        mlflow.log_param("learning_rate", settings.optimizer_kwargs.get("lr", None))
        
        config = ModelConfig(
            input_size=3,
            hidden_size=64,
            num_layers=1,
            output_size=20,
            dropout=0.1,
        )

        model = GRUmodel(
            config=config,
        )

        trainer = Trainer(
            model=model,
            settings=settings,
            loss_fn=loss_fn,
            optimizer=optim.Adam,
            traindataloader=trainstreamer,
            validdataloader=validstreamer,
            scheduler=optim.lr_scheduler.ReduceLROnPlateau,
            device=device,
        )
        trainer.loop()

        if not settings.earlystop_kwargs["save"]:
            tag = datetime.now().strftime("%Y%m%d-%H%M-")
            modelpath = modeldir / (tag + "model.pt")
            torch.save(model, modelpath)

2025/10/04 21:48:17 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/10/04 21:48:17 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
c:\Master Applied Data Science\Year 2\Semester 3 (Deep Learning & Model Deployment)\Portfolio-Marcello-Wienhoven\.venv\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


Repeat 1/3


2025-10-04 21:48:17.942 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to gestures\20251004-214817
2025-10-04 21:48:19.060 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 81/81 [00:01<00:00, 73.83it/s]
2025-10-04 21:48:20.396 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.9249 test 2.6532 metric ['0.1000']
100%|██████████| 81/81 [00:01<00:00, 76.11it/s]
2025-10-04 21:48:21.579 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 2.4070 test 2.2797 metric ['0.1938']
100%|██████████| 81/81 [00:00<00:00, 87.98it/s]
2025-10-04 21:48:22.635 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 2.2327 test 2.1162 metric ['0.2641']
100%|██████████| 81/81 [00:01<00:00, 78.34it/s]
2025-10-04 21:48:23.783 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 2.0016 test 1.9071 metric ['0.3125']
100%|██████████| 81/81 [00:00<00:00, 83.64it/s]
2025-10-04 21:48:24.87

Repeat 2/3


100%|██████████| 81/81 [00:01<00:00, 68.18it/s]
2025-10-04 21:48:38.140 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.9117 test 2.6322 metric ['0.1219']
100%|██████████| 81/81 [00:01<00:00, 75.92it/s]
2025-10-04 21:48:39.318 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 2.3699 test 2.2247 metric ['0.1984']
100%|██████████| 81/81 [00:00<00:00, 83.77it/s]
2025-10-04 21:48:40.425 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 2.1632 test 2.0271 metric ['0.2437']
100%|██████████| 81/81 [00:01<00:00, 69.32it/s]
2025-10-04 21:48:41.729 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 1.9608 test 1.8308 metric ['0.3594']
100%|██████████| 81/81 [00:00<00:00, 86.63it/s]
2025-10-04 21:48:42.804 | INFO     | mltrainer.trainer:report:209 - Epoch 4 train 1.7322 test 1.6244 metric ['0.4234']
100%|██████████| 81/81 [00:01<00:00, 77.99it/s]
2025-10-04 21:48:43.955 | INFO     | mltrainer.trainer:report:209 - Epoch 5 train 1.5354 test 1.7108 metric ['0.4109'

Repeat 3/3


100%|██████████| 81/81 [00:00<00:00, 83.91it/s]
2025-10-04 21:48:56.415 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.8891 test 2.4912 metric ['0.1313']
100%|██████████| 81/81 [00:00<00:00, 89.54it/s]
2025-10-04 21:48:57.433 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 2.3318 test 2.2004 metric ['0.2172']
100%|██████████| 81/81 [00:00<00:00, 90.27it/s]
2025-10-04 21:48:58.445 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 2.1247 test 1.9696 metric ['0.3578']
100%|██████████| 81/81 [00:00<00:00, 90.75it/s]
2025-10-04 21:48:59.452 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 1.8826 test 1.7443 metric ['0.3953']
100%|██████████| 81/81 [00:00<00:00, 85.13it/s]
2025-10-04 21:49:00.518 | INFO     | mltrainer.trainer:report:209 - Epoch 4 train 1.6227 test 1.4960 metric ['0.5422']
100%|██████████| 81/81 [00:00<00:00, 85.72it/s]
2025-10-04 21:49:01.580 | INFO     | mltrainer.trainer:report:209 - Epoch 5 train 1.4042 test 1.3179 metric ['0.5891'

In [12]:
mlflow.end_run()

## Open MLFlow
"""
```bash
mlflow server \
    --backend-store-uri sqlite:///mlflow.db \
    --host 127.0.0.1 \ 
    --port 5000 \
        
mlflow server --backend-store-uri sqlite:///3-hypertuning-rnn/mlflow.db --host 127.0.0.1 --port 5000
```
"""